# ${\color{brown}\mathbb{NAMES:}}$
1- ${\color{yellow}Yuval\ Kaver}$, id: 329

2- ${\color{yellow}Matan\ Ginzburg}$, id: 215

# Point Selection

In [29]:
import matplotlib
%matplotlib tk

import matplotlib.pyplot as plt
from skimage import io 
import skimage
import numpy as np
import scipy
from numpy.linalg import inv 

In [30]:
im = io.imread('C:\\Users\\matan\\Documents\\Visual Code thong\\GEOMETRY DASH רבת מבטים\\rubik-cube-01.jpg')
im.shape

(1080, 1920, 3)

# The 6 Points Algorithm

In [31]:
''' Synthetic test. '''
K = np.array([
    [150, 0, 23],
    [0, 95, 4],
    [0, 0, 1]
])
R = np.array([[ 1,0, 0],
                  [0,0.5,-0.8660254],   
                  [0,0.8660254,0.5]])

t = np.array([1, 5, 2]).reshape(3, 1)

M_synthetic = np.hstack(( K@R , K@t ))

n = 10
def generate_synthetic_points():
    points = abs(np.random.randn(n,3))
    points = np.hstack((points,np.ones((n,1))))
    pixels = np.zeros((n,3))
    for i in range(n):
        pixels[i] = M_synthetic@points[i].T
        pixels[i] = pixels[i]/pixels[i][2]
    pixels_with_error = pixels + np.hstack((np.random.randn(n,2),np.zeros((n,1))))
    return points,pixels_with_error
    return points,pixels
generate_synthetic_points()

(array([[0.89455, 1.69146, 0.13988, 1.     ],
        [0.29041, 0.19904, 1.19619, 1.     ],
        [0.4403 , 0.31501, 1.94818, 1.     ],
        [0.59642, 0.58007, 0.55161, 1.     ],
        [1.43662, 0.4887 , 1.2373 , 1.     ],
        [1.69884, 0.12719, 0.19581, 1.     ],
        [2.38298, 1.03941, 1.43211, 1.     ],
        [1.12265, 1.17104, 1.57846, 1.     ],
        [1.695  , 0.94344, 1.12324, 1.     ],
        [0.65663, 1.14373, 0.61426, 1.     ]]),
 array([[101.24758, 159.2459 ,   1.     ],
        [ 93.57495, 141.74778,   1.     ],
        [ 90.87435, 105.11871,   1.     ],
        [108.33163, 168.60563,   1.     ],
        [140.93929, 133.62801,   1.     ],
        [206.67825, 214.70986,   1.     ],
        [162.74112, 116.57325,   1.     ],
        [107.10727, 108.51442,   1.     ],
        [144.01141, 129.86651,   1.     ],
        [ 97.82671, 148.09408,   1.     ]]))

In [32]:
def rubix_points():
    P = np.array([[0,0,0],[0,1,0],
                [1,0,1],[2,0,2],
                [0,0,3],[1,1,3],
                [3,0,3],[3,0,1],
                [3,3,3],[0,3,0],
                [3,0,0],[0,3,3],
                [1,3,3]])
    p = np.array([(np.float64(851.1129032258063), np.float64(946.5903225806452)),
                (np.float64(1029.1774193548388), np.float64(861.4290322580645)),
                (np.float64(715.6290322580644), np.float64(679.4935483870968)),
                (np.float64(591.758064516129), np.float64(396.9129032258063)),
                (np.float64(839.5), np.float64(327.2354838709675)),
                (np.float64(885.9516129032259), np.float64(199.49354838709655)),
                (np.float64(479.50000000000006), np.float64(145.29999999999973)),
                (np.float64(514.3387096774193), np.float64(532.3967741935483)),
                (np.float64(973.8856143856142), np.float64(34.025474525474465)),
                (np.float64(1339.2802197802196), np.float64(746.1133866133866)),
                (np.float64(530.8086913086913), np.float64(690.0094905094904)),
                (np.float64(1365.9516129032259), np.float64(185.69354838709683)),
                (np.float64(1224.1953046953045), np.float64(124.65484515484513))])
    P = np.hstack((P,np.ones((13,1))))
    p = np.hstack((p,np.ones((13,1))))
    values = np.arange(4)
    # Create a grid of values for x, y, z
    x, y, z = np.meshgrid(values, values, values, indexing='ij')
    # Stack the grids along a new axis to get combinations
    combinations = np.stack((x, y, z), axis=-1)
    # Reshape to a 2D array where each row is a combination
    combinations = combinations.reshape(-1, 3)
    # Create a boolean mask for the condition: x and y are non-zero and z < 3
    mask = ~((combinations[:, 0] > 0) & (combinations[:, 1] > 0) & (combinations[:, 2] < 3))
    # Apply the mask to filter out the unwanted combinations
    filtered_combinations = combinations[mask]

    P_rubix = np.hstack((filtered_combinations,np.ones((37,1))))
    np.shape(P_rubix)
    return P,p,P_rubix

In [33]:
def reprojection_error(M,P,p):
    A = np.reshape(M,(3,4))
    Error = 0    
    for i in range(len(P)):

        u_i = p[i][0]
        v_i = p[i][1]
        
        A_pi_x = (A[0].T@P[i])/(A[2].T@P[i])
        A_pi_y = (A[1].T@P[i])/(A[2].T@P[i])
        
        Error += (A_pi_x - u_i)**2 + (A_pi_y - v_i)**2

    return Error


In [34]:
def linear_calibration(p,P):
    '''
    Input: 
        p: nx2 array (pixels in an image)
        P: nx3 array (points in 3d)
        Assumption: The two arrays have corresponding order. 
    Output: 
        M: camera matrix 
    '''
    A = np.zeros((2*len(P),12))
    for i in range(len(P)):
        A[2*i] = [-P[i][0],-P[i][1],-P[i][2],-1,0,0,0,0,P[i][0]*p[i][0],P[i][1]*p[i][0],P[i][2]*p[i][0],p[i][0]]
        A[2*i+1] = [0,0,0,0,-P[i][0],-P[i][1],-P[i][2],-1,P[i][0]*p[i][1],P[i][1]*p[i][1],P[i][2]*p[i][1],p[i][1]]
    U , _ , __= np.linalg.svd(A.T@A)
    M1 = np.reshape(U.T[-1],(3,4))
    return M1


In [35]:
'''synthetic test for linear calibration'''
P, p =  generate_synthetic_points()
M_got = linear_calibration(p,P)
np.set_printoptions(suppress=True,precision=5)    
print(f"the M we got:\n{(M_got*M_synthetic[2,3]/M_got[2,3])}\nthe real M:\n{M_synthetic}")
print("reprojection error:",reprojection_error(M_got,P,p))

the M we got:
[[141.48851  12.90867  18.04389 196.5045 ]
 [ -8.58257  41.65228 -74.66127 482.70167]
 [ -0.05413   0.80188   0.53597   2.     ]]
the real M:
[[150.       19.91858  11.5     196.     ]
 [  0.       50.9641  -80.27241 483.     ]
 [  0.        0.86603   0.5       2.     ]]
reprojection error: 3.0550158474337343


In [36]:
''' Here we test the calibration with the points selected above in the real image.'''
plt.imshow(im)
P,p,P_rubix = rubix_points()
M_got = linear_calibration(p,P)
pixels_got = np.zeros((37,3))
for i in range(37):
    pixels_got[i] = M_got@P_rubix[i]
    pixels_got[i] = pixels_got[i]/pixels_got[i][2]
for i in pixels_got:
    plt.plot(i[0], i[1], 'bo') 
    
    

In [37]:
def geometric_calibration_refinement(p,P,M):
    '''    
    Input: 
        p: nx2 array (pixels in an image)
        P: nx3 array (points in 3d)
        Assumption: The two arrays have corresponding order. 
        M: first guess of the camera matrix.
        The functions refines M by minimizing the reprojecion error.
    Output:
        M (refined)
    '''
    M_flat = np.matrix.flatten(M)
    M_min = scipy.optimize.least_squares(reprojection_error,x0=M_flat,args=(P,p))
    return M_min.x.reshape((3,4))




In [38]:
'''synthetic test for refinement'''
P, p =  generate_synthetic_points()
M_got = linear_calibration(p,P)
M_refined = np.reshape(geometric_calibration_refinement(p,P,M_got),(3,4))
M_refined = M_refined*M_synthetic[2,3]/M_refined[2,3]
M_got = M_got*M_synthetic[2,3]/M_got[2,3]
np.set_printoptions(suppress=True,precision=5)
print(f"the original M we got:\n{(M_got*M_synthetic[2,3]/M_got[2,3])}\nthe real M:\n{M_synthetic}")
print("reprojection error:",reprojection_error(M_got,P,p))
print(f"the refined M:\n{M_refined}\nthe real M:\n{M_synthetic}")
print("reprojection error:",reprojection_error(M_refined,P,p))


the original M we got:
[[135.49294   3.83358  15.51596 197.31143]
 [ -9.89065  18.85976 -71.5019  479.97805]
 [ -0.05693   0.6865    0.52428   2.     ]]
the real M:
[[150.       19.91858  11.5     196.     ]
 [  0.       50.9641  -80.27241 483.     ]
 [  0.        0.86603   0.5       2.     ]]
reprojection error: 6.0387353076363555
the refined M:
[[135.44874   3.83233  15.51091 197.24704]
 [ -9.88744  18.8536  -71.47858 479.82143]
 [ -0.05682   0.68594   0.52319   2.     ]]
the real M:
[[150.       19.91858  11.5     196.     ]
 [  0.       50.9641  -80.27241 483.     ]
 [  0.        0.86603   0.5       2.     ]]
reprojection error: 6.0307384954819


In [39]:
'''Here you must test the calibration with the points selected above in the real image.'''
plt.imshow(im)
P,p,P_rubix = rubix_points()
M_got = linear_calibration(p,P)
M_refined = geometric_calibration_refinement(p,P,M_got)
pixels_got = np.zeros((37,3))
for i in range(37):
    pixels_got[i] = M_refined@(P_rubix[i])
    pixels_got[i] = pixels_got[i]/pixels_got[i][2]
for i in pixels_got:
    plt.plot(i[0], i[1], 'ro') 

# Single View Geometry

In [40]:
#syntetic M:

K = np.array([
    [8, 0, 32],
    [0, 8, 24],
    [0, 0, 1]
])
R = np.array([[ 1,0, 0],            #its ortoghonal! so good...
                  [0,0.5,-0.8660254],   
                  [0,0.8660254,0.5]])

t = np.array([1, 5, 2]).reshape(3, 1)

M_syn = np.hstack(( K@R , K@t ))
print("M = \n",M_syn)

M = 
 [[ 8.      27.71281 16.      72.     ]
 [ 0.      24.78461  5.0718  88.     ]
 [ 0.       0.86603  0.5      2.     ]]


In [41]:
def camera_decomposition(M):
    '''
    Input: M (camera matrix)
    Output: K,R,t 
    '''             
    m_bar = np.delete(M,3,1)                # now we have : M = [m_bar(3x3),m_small(3x1)] which is 4x3! 
    m_small = np.delete(M,(0,1,2),1) 
    
    M_MT = m_bar @ m_bar.T
    
    P = np.array(
        [[0,0,1],
        [0,1,0],
        [1,0,0]])

    L = np.linalg.cholesky(P@M_MT@P,upper=False)
    our_lambda = L[2,2]
    K = P@L@P
    R =  1/our_lambda * np.linalg.inv(K) @ m_bar
    t = 1/our_lambda * np.linalg.inv(K) @ m_small



    return K,R,t
# print(K@K.T)
k,r,t_got = camera_decomposition(M_syn)
np.set_printoptions(suppress=True,precision=5)
print("the real K:\n",K)
print("the k we got:\n",(k/k[2,2]))
print("the real R:\n",R)
print("the R we got:\n",(r*R[2,2]/r[2,2]))
print("the real t:\n",t)
print("the t we got:\n",(t_got*t[2]/t_got[2]))


the real K:
 [[ 8  0 32]
 [ 0  8 24]
 [ 0  0  1]]
the k we got:
 [[ 8. -0. 32.]
 [ 0.  8. 24.]
 [ 0.  0.  1.]]
the real R:
 [[ 1.       0.       0.     ]
 [ 0.       0.5     -0.86603]
 [ 0.       0.86603  0.5    ]]
the R we got:
 [[ 1.       0.      -0.     ]
 [ 0.       0.5     -0.86603]
 [ 0.       0.86603  0.5    ]]
the real t:
 [[1]
 [5]
 [2]]
the t we got:
 [[1.]
 [5.]
 [2.]]


In [42]:
def camera_center(M):
    omega = scipy.linalg.null_space(M)
    return omega/omega[3]
print("we got:\n",camera_center(M_syn))
real_center = np.vstack((-R.T@t,np.ones((1,1))))
print("real center is:\n",real_center)

we got:
 [[-1.     ]
 [-4.23205]
 [ 3.33013]
 [ 1.     ]]
real center is:
 [[-1.     ]
 [-4.23205]
 [ 3.33013]
 [ 1.     ]]


In [43]:
def infinity_point_optical_ray(M,p):
    '''
    Returns the point at infinity of the optical ray generated by the pixel p for the given camera matrix M. 
    '''
    K, R, t = camera_decomposition(M)
    p_infinity = np.array([R.T@inv(K)@p])
    p_infinity = np.hstack((p_infinity,np.zeros((1,1))))
    return p_infinity
pp = np.array([5,64,1])
real_inf=(R.T@inv(K)@pp)
real_inf = np.hstack((real_inf,0))
print("real point is:\n",real_inf)
inf_got = infinity_point_optical_ray(M_syn,pp)
print("we got:\n",inf_got*real_inf[2]/inf_got[0][2])


real point is:
 [-3.375    3.36603 -3.83013  0.     ]
we got:
 [[-3.375    3.36603 -3.83013  0.     ]]


In [44]:
def focal_plane(M,l):
    '''
    Returns the vector representing the plane generated the line l (in the image) for the given camera matrix M. 
    '''
    return M.T@l 

pixel1 = np.array([6,4,1])
pixel2 = np.array([7,5,1])
l = pixel1 - pixel2
focal_p = focal_plane(M_syn,l)
omega = camera_center(M_syn)
print(focal_p)
print(f"(M.T@L).T@omega = {focal_p.T@omega}")

[  -8.       -52.49742  -21.0718  -160.     ]
(M.T@L).T@omega = [0.]


# Image Warping

In [45]:
'''
Purpose: Take the image of the floor. Compute an homography to make the square a real square and perform an image warping using scikit-image.
Display the original and the destination images.  
'''

'\nPurpose: Take the image of the floor. Compute an homography to make the square a real square and perform an image warping using scikit-image.\nDisplay the original and the destination images.  \n'

In [46]:
# here we wrote the functions from last exercise
def svd_computation_homography(p: np.ndarray, q: np.ndarray):
    '''
    Given two nx2 arrays of affine points, the function computes the homography between these two sets of points by the SVD method. 
    Output: H a 3x3 matrix
    '''
    M = np.zeros((2*len(p),9))
    for i in range(len(p)):
        M[2*i] = [p[i][0]*1,p[i][1]*1,1*1,0,0,0,-p[i][0]*q[i][0],-p[i][1]*q[i][0],-1*q[i][0]]
        M[2*i+1] = [0,0,0,p[i][0],p[i][1],1,-p[i][0]*q[i][1],-p[i][1]*q[i][1],-q[i][1]]
    MTM = M.T.dot(M)
    U , _ , __= np.linalg.svd(MTM)
    A = np.reshape(U[:,-1],(3,3))
    return A
def normalization(p: np.ndarray):
    '''
    Given a nx2 array of affine points, the function computes the normalizing homography of these points.
    Output: T a 3x3 matrix (which lies in Aff2).
    '''
    avg_x, avg_y, _ = np.mean(p,axis=0)
    std_x, std_y, _ = np.std(p,axis=0)
    T = np.array([
        [1/std_x,0,-avg_x/std_x],
        [0,1/std_y,-avg_y/std_y],
        [0,      0,          1]
    ])
    return T
def homography(p: np.ndarray, q: np.ndarray):
    '''
    Given two nx2 arrays of affine points, the function computes the homography between these two sets of points by normalizations and the SVD method. 
    Output: H a 3x3 matrix
    '''
    T1 = normalization(p)
    T2 = normalization(q)
    p_normalized = np.array(p)
    q_normalized = np.array(p)
    for i in range(len(p)):
        p_normalized[i] = T1@p[i] 
        q_normalized[i] = T2@q[i]    
    B = svd_computation_homography(p_normalized,q_normalized)
    return (inv(T2)).dot(B.dot(T1))

In [47]:
# setting points
p = np.array([[383.61916544477026, 406.1280241935483],
 [470.94275415444764, 272.93749999999994],
 [740.852028347996, 324.9788306451612],
 [737.3238025415444, 489.92338709677415]])

P = np.array([[400,400],
              [800,400],
              [800,800],
              [400,800]])
#------------------------------------------------------

p = np.hstack((p,np.ones((4,1))))
P = np.hstack((P,np.ones((4,1))))
homography_of_floor = homography(P,p)

floor_image = plt.imread ("\\Users\\matan\\Documents\\Visual Code thong\\GEOMETRY DASH רבת מבטים\\Castellanparquet_Brun4.jpg")

In [48]:
'''warping!'''
squares = skimage.transform.warp(floor_image ,homography_of_floor,output_shape=(800, 1215))         
     
fig, ax = plt.subplots(nrows=2)
ax[0].imshow(floor_image)
ax[1].imshow(squares)

for a in ax:
    a.axis('off')

#plt.show()
plt.tight_layout()